In [39]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import Random Forest Model classifier
from sklearn.ensemble import RandomForestClassifier
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import seaborn as sns
pd.set_option('max_columns', None)
%matplotlib inline

In [40]:
# Open calendar data
boston_calendar = pd.read_csv("boston_airbnb_data/calendar.csv")
seattle_calendar = pd.read_csv("seattle_airbnb_data/calendar.csv")
# Open listings data
boston_listings = pd.read_csv("boston_airbnb_data/listings.csv")
seattle_listings = pd.read_csv("seattle_airbnb_data/listings.csv")

In [41]:
# Concatenate the data calendar from Boston and Seattle
df_calendar = pd.concat([boston_calendar,seattle_calendar ], axis=0,ignore_index=True)
# Drop price column because we don't need it for our analysis
df_calendar = df_calendar.drop('price', axis=1)
# Get one column for each variable on for t(true) and one for f(false)
df_calendar = pd.concat([df_calendar.drop('available', axis=1), pd.get_dummies(df_calendar['available'], prefix='available', prefix_sep='_')], axis=1)
# Group by each list id by adding the number of times each list_id is available and unavailable
df_occupation = df_calendar.groupby("listing_id").sum()
# Add a column with the occupancy percentage, which is the number of days occupied divided by the total number of days registered
df_occupation["occupation_percentage"] = df_occupation["available_f"]*100/(df_occupation["available_f"]+df_occupation["available_t"])
# Function Transform the output variable occupation_percentage that is continuous to categorical with five levels
def percentage_to_categorical(value):
    value = float(value)
    if value == 0:
        return 1
    elif value > 0 and value <= 100:
        return math.ceil(value/20)   
    else:
        return np.nan
# Apply function
df_occupation["occupation_percentage"] = df_occupation["occupation_percentage"].apply(percentage_to_categorical)
# Drop columns will no longer be used
df_occupation = df_occupation.drop(["available_f","available_t"], axis=1)
# Before concatenating listings dataframes, we need to remove the columns that have boston and not seattle
boston_listings = boston_listings.drop( ['access', 'interaction', 'house_rules'], axis=1)
# Concatenate the data listings from Boston and Seattle
df_listings = pd.concat([boston_listings, seattle_listings], axis=0)

In [42]:
# Create a consolidate dataframe with all the airbnb data including the ocupattion rate that wass alcasdflklasdf
df = pd.merge(df_listings, df_occupation,left_on="id",right_on="listing_id", how="inner")

In [43]:
# cambiar a diccionario con su dimension de valor
columns_for_analysis = ['host_location','host_response_time','host_response_rate','host_acceptance_rate','host_is_superhost',
                        'host_total_listings_count','host_verifications','host_has_profile_pic','host_identity_verified',
                        'neighbourhood_cleansed','city','zipcode','latitude','longitude','property_type','room_type',
                        'accommodates','bathrooms','bedrooms','beds','bed_type','amenities','square_feet','price',
                        'security_deposit','cleaning_fee','guests_included','extra_people','minimum_nights',
                        'maximum_nights','number_of_reviews','review_scores_rating','review_scores_accuracy',
                        'review_scores_cleanliness','review_scores_checkin','review_scores_communication',
                        'review_scores_location','review_scores_value','requires_license','instant_bookable',
                        'cancellation_policy','require_guest_profile_picture','require_guest_phone_verification',
                        'reviews_per_month','occupation_percentage']

In [44]:
df = df[columns_for_analysis]

In [45]:
df = df[df["city"].isin(['Boston','Seattle'])]

In [46]:
# Create column host_isin_city
def host_isin_city(host_location,city):
    try:
        clean_host_location = host_location.split(",")[0]
        if clean_host_location == city:
            return 1
        else:
            return 0
    except AttributeError:
        return np.nan

df['host_in_city'] = df.apply(lambda x: host_isin_city(x.host_location, x.city), axis=1)
df = df.drop(["host_location"],axis=1)

In [47]:
# Create a dataframe with the column name and the percent of missing values per column
percent_missing = df.isnull().sum()*100/len(df)

df_nullrows_percent = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing}).reset_index(drop=True)

In [48]:
df_nullrows_percent[df_nullrows_percent["percent_missing"]>0].sort_values(by=['percent_missing'],ascending=False)

,column_name,percent_missing
21,square_feet,97.955778
23,security_deposit,56.681964
24,cleaning_fee,28.633013
31,review_scores_accuracy,19.774718
33,review_scores_checkin,19.733000
36,review_scores_value,19.719093
35,review_scores_location,19.719093
32,review_scores_cleanliness,19.649562
34,review_scores_communication,19.621749
30,review_scores_rating,19.496593


In [49]:
df = df.drop(["square_feet","security_deposit","cleaning_fee"],axis=1)

In [50]:
# Extract the % sign and transform the value to float.
df['host_response_rate'] = df['host_response_rate'].str.replace('%', '', regex=False).astype(float)
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.replace('%', '', regex=False).astype(float)
# Extract the $ and "," sign and transform the value to float
df['price'] = df['price'].str.replace('$', '', regex=False)
df['price'] = df['price'].str.replace(',', '', regex=False)
df['price'] = df['price'].astype(float)
df['extra_people'] = df['extra_people'].str.replace('$', '', regex=False)
df['extra_people'] = df['extra_people'].str.replace(',', '', regex=False)
df['extra_people'] = df['extra_people'].astype(float)

In [51]:
fill_mean_cols = ["review_scores_accuracy","review_scores_location","review_scores_value","review_scores_checkin",
                 "review_scores_communication","review_scores_cleanliness","review_scores_rating","reviews_per_month",
                 "host_response_rate","host_acceptance_rate","host_response_rate","host_acceptance_rate"]

fill_mean = lambda col: col.fillna(col.mean())
df[fill_mean_cols] = df[fill_mean_cols].apply(fill_mean)

In [52]:
var = "host_response_time"
df = pd.concat([df.drop(var, axis=1), pd.get_dummies(df[var],prefix=var, prefix_sep='-', drop_first=True,dummy_na=True)], axis=1)

In [53]:
df = df.dropna()

In [54]:
# function to clean columns with str values with list or dict format
def get_dummies_strlist(df,column_name):

    # Creat a list to save all possible values
    all_items = []
    # Characters to be remove
    replacements = ['"',"'","[","]","{","}"]
    # Loop through all rows in the df
    for index, row in df.iterrows():
        # Select the value based on the column being cleaned
        items = row[column_name]
        # Loop through the characters that need to be removed
        for char in replacements:
            if char in items:
                # Remove the character
                items = items.replace(char,"")
        # Convert already cleaned str value to list
        items = items.split(",")
        # Loop through the items in the list
        for item in items:
            # Check that the item has at least one character
            if len(item) != 0:
                # Clears the item in case it has empty spaces
                clean_item = item.strip()
                clean_item = "{}-{}".format(column_name,clean_item)
                # Check that a column already exists for that item
                if clean_item in df.columns:
                    # If exists assign a value of 1 to that row and column
                    df.loc[index, clean_item] = 1
                else:
                    # If it does not exist create the column and then assign the value of 1 to that row and column
                    df[clean_item] = 0
                    df.loc[index, clean_item] = 1
                    
    # Drop the original column that was transformed
    df = df.drop([column_name], axis=1)
    # Return the new dataframe
    return df

In [55]:
df = get_dummies_strlist(df,"host_verifications")

In [29]:
df = get_dummies_strlist(df,"amenities")

In [56]:
df_exp1 = df.drop(['zipcode','neighbourhood_cleansed','city','amenities'],axis=1)

In [57]:
# Create dummi variables for the other categorical columns
cat_vars = df_exp1.select_dtypes(include=['object']).copy().columns
for var in  cat_vars:
    # for each cat add dummy var, drop original column
    df_exp1 = pd.concat([df_exp1.drop(var, axis=1), pd.get_dummies(df_exp1[var],prefix=var, prefix_sep='-', drop_first=True)], axis=1)

In [58]:
df_exp1

,host_response_rate,host_acceptance_rate,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,occupation_percentage,host_in_city,host_response_time-within a day,host_response_time-within a few hours,host_response_time-within an hour,host_response_time-nan,host_verifications-email,host_verifications-phone,host_verifications-facebook,host_verifications-reviews,host_verifications-linkedin,host_verifications-amex,host_verifications-jumio,host_verifications-kba,host_verifications-manual_online,host_verifications-manual_offline,host_verifications-google,host_verifications-sent_id,host_verifications-weibo,host_verifications-photographer,host_is_superhost-t,host_has_profile_pic-t,host_identity_verified-t,property_type-Bed & Breakfast,property_type-Boat,property_type-Bungalow,property_type-Cabin,property_type-Camper/RV,property_type-Chalet,property_type-Condominium,property_type-Dorm,property_type-Entire Floor,property_type-Guesthouse,property_type-House,property_type-Loft,property_type-Other,property_type-Tent,property_type-Townhouse,property_type-Treehouse,property_type-Villa,property_type-Yurt,room_type-Private room,room_type-Shared room,bed_type-Couch,bed_type-Futon,bed_type-Pull-out Sofa,bed_type-Real Bed,instant_bookable-t,cancellation_policy-moderate,cancellation_policy-strict,cancellation_policy-super_strict_30,require_guest_profile_picture-t,require_guest_phone_verification-t
0,94.897477,92.005022,1.0,42.282619,-71.133068,4,1.5,2.0,3.0,250.0,1,0.0,2,1125,0,93.352565,9.543248,9.421945,9.721587,9.735813,9.52503,9.322882,2.023588,5,1.0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1,100.000000,100.000000,1.0,42.286241,-71.134374,2,1.0,1.0,1.0,65.0,0,0.0,2,15,36,94.000000,10.000000,9.000000,10.000000,10.000000,9.00000,9.000000,1.300000,1,1.0,0,0,1,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0
2,100.000000,88.000000,1.0,42.292438,-71.135765,2,1.0,1.0,1.0,65.0,1,20.0,3,45,41,98.000000,10.000000,9.000000,10.000000,10.000000,9.00000,10.000000,0.470000,1,1.0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0
4,100.000000,100.000000,1.0,42.284512,-71.136258,2,1.5,1.0,2.0,79.0,1,0.0,2,31,29,99.000000,10.000000,10.000000,10.000000,10.000000,9.00000,10.000000,2.250000,1,1.0,0,0,1,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
5,100.000000,95.000000,2.0,42.291690,-71.131893,2,1.0,1.0,1.0,75.0,1,0.0,2,1125,8,100.000000,10.000000,10.000000,10.000000,10.000000,9.00000,10.000000,1.700000,5,1.0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7398,99.000000,100.000000,354.0,47.664295,-122.359170,6,2.0,3.0,3.0,359.0,1,0.0,3,1125,1,80.000000,8.000000,10.000000,4.000000,8.000000,10.00000,8.000000,0.300000,5,0.0,0,1,0,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
7399,100.000000,100.000000,1.0,47.649552,-122.318309,4,1.0,1.0,2.0,79.0,3,25.0,2,29,2,100.000000,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000,2.000000,2,1.0,0,0,1,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
7400,94.897477,92.005022,1.0,47.508453,-122.240607,2,1.0,1.0,1.0,93.0,2,20.0,1,7,0,93.352565,9.543248,9.421945,9.721587,9.735813

In [59]:
#Split data into an X matrix and a response vector y
y = df_exp1['occupation_percentage']
x = df_exp1.drop('occupation_percentage', axis=1)

# Split dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [60]:
#Create a Gaussian Classifier
rfc=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
rfc.fit(x_train,y_train)

y_pred=rfc.predict(x_test)

In [61]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5710922787193974


In [62]:
feature_importance = pd.Series(rfc.feature_importances_,index=list(x.columns)).sort_values(ascending=False)

In [63]:
dict(feature_importance)

{'latitude': 0.08272727265181838,
 'longitude': 0.08075809407608024,
 'price': 0.06243207264519257,
 'number_of_reviews': 0.056293834470455564,
 'reviews_per_month': 0.05477515445731741,
 'host_total_listings_count': 0.05049679093289488,
 'review_scores_rating': 0.03628720493110802,
 'maximum_nights': 0.03174452984194395,
 'extra_people': 0.03010387901968357,
 'accommodates': 0.029717474357011355,
 'minimum_nights': 0.029580490515938983,
 'host_acceptance_rate': 0.02839649477770295,
 'host_response_rate': 0.026772067521427755,
 'guests_included': 0.0200008460043152,
 'bedrooms': 0.0198871832659187,
 'beds': 0.018669535139085657,
 'review_scores_value': 0.016717600247144734,
 'review_scores_cleanliness': 0.01643413305470408,
 'review_scores_location': 0.015006155541858524,
 'bathrooms': 0.014627443725986402,
 'review_scores_accuracy': 0.013780540739133219,
 'cancellation_policy-moderate': 0.013413769712308544,
 'host_verifications-kba': 0.013300138097172218,
 'host_verifications-jumio':